# Detailed Analysis (Bid by Bid, Card by Card)

This is a tutorial of how to do a detailed analysis of a played board.

The engine looks at the bidding and play as it originally happened, and does an analysis for every bid and every card played.

The analysis is not just a double-dummy analysis for the exact current layout (like if you would press the "GIB" button on BBO). Instead, it's an analysis over many different possible layouts (samples).

In [1]:
import os
os.chdir('..')

from nn.models import Models
from analysis import CardByCard
from util import parse_lin, display_lin
from sample import Sample
import conf

In [2]:
models = Models.from_conf(conf.load('./config/default.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('./config/default.conf'))  # Load sampling strategies

Instructions for updating:
non-resource variables are not supported in the long term
INFO:tensorflow:Restoring parameters from ..\UCBC 2024/Models/NS1EW99-bidding_same-5556000
INFO:tensorflow:Restoring parameters from ..\UCBC 2024/Models/NS1EW99-binfo_same-5556000
INFO:tensorflow:Restoring parameters from ..\UCBC 2024/Models/lead_suit-999000
INFO:tensorflow:Restoring parameters from ..\UCBC 2024/Models/lead_nt-475000
INFO:tensorflow:Restoring parameters from ..\UCBC 2024/Models/Jack/lr3-1000000
INFO:tensorflow:Restoring parameters from ..\UCBC 2024/Models/single_dummy-32768000
INFO:tensorflow:Restoring parameters from ..\UCBC 2024/Models/lefty_nt-475000
INFO:tensorflow:Restoring parameters from ..\UCBC 2024/Models/dummy_nt-475000
INFO:tensorflow:Restoring parameters from ..\UCBC 2024/Models/righty_nt-475000
INFO:tensorflow:Restoring parameters from ..\UCBC 2024/Models/decl_nt-950000
INFO:tensorflow:Restoring parameters from ..\UCBC 2024/Models/lefty_suit-999000
INFO:tensorflow:Restorin

In [3]:
# we specify all the information about a board
# (it's quite tedious to enter every single thing by hand here,
# later we'll have an example of how you can give it a board played on BBO)

dealer = 'S'
vuln = [True, True]  # fist element is NS, second element is EW

hands = [
    'AJ87632.J96.753.',
    'K9.Q8542.T6.AJ74',
    'QT4.A.KJ94.KQ986',
    '5.KT73.AQ82.T532'
]

auction = ['1N', 'PASS', '4H', 'PASS', '4S', 'PASS', 'PASS', 'PASS']

play = ['C2', 'D3', 'CA', 'C6', 'D6', 'DJ', 'DQ', 'D5', 'DA', 'D7', 'DT', 'D4', 'D8', 'H6', 'H2', 'D9', 'SQ', 'S5', 'S2', 'SK', 'H4', 'HA', 'H7', 'H9', 'S4', 'C3', 'SA', 'S9', 'S3', 'C4', 'ST', 'H3', 'CK', 'C5', 'HJ', 'C7', 'C8', 'CT', 'S6', 'CJ', 'S7', 'H8', 'C9', 'D2', 'S8', 'H5', 'CQ', 'HT', 'SJ', 'HQ', 'DK', 'HK']

In [4]:
card_by_card = CardByCard(dealer, vuln, hands, auction, play, models, sampler, False)

In [5]:
# calling this starts the analysis
# it will go bid-by-bid and card-by-card, and will take a few moments
# possible mistakes will be annotated with ? or with ?? (if it's a bigger mistake)
# (possible mistake means that the engine does not agree with the bid/play. the engine could be wrong too :))

await card_by_card.analyze()

analyzing the bidding
1N Suggested bid from NN: CandidateBid(bid=1C  , insta_score=0.9993, expected_score=---, adjust=---)
1N is not in the bids from the neural network
PASS OK NN-value: 1.000


Loaded lib dds.dll


4H OK NN-value: 0.493
PASS Suggested bid from NN: CandidateBid(bid=X   , insta_score=0.7866, expected_score=-243.75, adjust=  39)
PASS NN-values:CandidateBid(bid=PASS, insta_score=0.2034, expected_score=-259.05, adjust=  10)
4S OK NN-value: 0.193
PASS OK NN-value: 1.000
PASS OK NN-value: 0.934
PASS OK NN-value: 0.996
analyzing opening lead
C2
C2 OK
analyzing play
Threads are still running after 3.0 second. Examined 318 of 5200300
Playouts: 317
D3 ? losing: 0.27
CA ?? losing: 0.96
Threads are still running after 3.0 second. Examined 734 of 2704156
Playouts: 613
C6 OK
Threads are still running after 3.0 second. Examined 142 of 2704156
Playouts: 124
D6 OK
DJ OK
Threads are still running after 3.0 second. Examined 1939 of 1352078
Playouts: 1611
DQ OK
D5 OK
Threads are still running after 3.0 second. Examined 153 of 705432
Playouts: 139
DA OK
D7 OK
Threads are still running after 3.0 second. Examined 1539 of 352716
Playouts: 1254
DT OK
D4 OK
Threads are still running after 3.0 second. Exami

In [6]:
# the engine does not agree with the 1N opening.
# indeed, it's a little offbeat with a singleton
# let's see what the engine is thinking (what would it bid instead)

card_by_card.bid_responses[0].to_dict()  # the 0 index is the first bid in the auction

{'bid': '1N',
 'who': 'Analysis',
 'candidates': [{'call': '1C', 'insta_score': 0.999}],
 'hcp': [3.1, 4.0, 3.1, 2.7, 3.2, 3.9, 3.0, 2.7, 3.2, 3.8, 3.0, 2.8],
 'shape': [8.0, 8.6, 7.8]}

the engine very confidently opens `1C` and doesn't even consider `1N`

In [7]:
# what about the opening lead? let's see...

card_by_card.cards['C2'].to_dict()

{'card': 'C2',
 'quality': 'Good',
 'hcp': [6.1, 2.3, 2.2, 2.2, 2.5, 3.5, 3.5, 3.5, 3.1, 3.2, 3.3, 3.3],
 'shape': [7.2, 8.1, 15.9],
 'candidates': [{'card': 'Hx',
   'insta_score': 0.179,
   'expected_tricks_sd': 10.41,
   'p_make_contract': 0.27},
  {'card': 'Cx',
   'insta_score': 0.231,
   'expected_tricks_sd': 10.34,
   'p_make_contract': 0.25},
  {'card': 'Dx',
   'insta_score': 0.175,
   'expected_tricks_sd': 10.76,
   'p_make_contract': 0.19}],
 'samples': ['x.KTxx.AQ8x.Txxx T98xxxx.8x.Kxx.K AQ.Q9xx.9xx.Q98x KJx.AJx.JTx.AJxx 0.86412',
  'x.KTxx.AQ8x.Txxx T98xxxx.AQx..KQ8 x.98xx.JT9xx.Axx AKQJ.Jx.Kxxx.J9x 0.74574',
  'x.KTxx.AQ8x.Txxx AQJ9xxx.Ax.xx.9x 8x.98xx.JT9.KJ8x KTx.QJx.Kxxx.AQx 0.74762',
  'x.KTxx.AQ8x.Txxx KQT8xx.AQJx.xx.Q Jx.98x.T9xx.98xx A9xx.xx.KJx.AKJx 0.74293',
  'x.KTxx.AQ8x.Txxx QT98xx.A9.Jx.AJx Jxx.Q8xxx.9xx.9x AKx.Jx.KTxx.KQ8x 0.73799',
  'x.KTxx.AQ8x.Txxx KJ98xxx.Ax.Jx.Jx Tx.98xx.KTxx.K9x AQx.QJx.9xx.AQ8x 0.74900',
  'x.KTxx.AQ8x.Txxx KT98xxx.xxx.KT.J Jx.AQ98.9

the engine agrees with leading a low club, but it's very close. the alternative is a low heart

In [8]:
# the engine considers dummy's discard of D3 on the first trick a big mistake.
# perhaps we should ruff instead, let's see what the engine suggests

card_by_card.cards['D3'].to_dict()

{'card': 'S7',
 'quality': 'Good',
 'hcp': [8.9, 8.8],
 'shape': [1.8, 3.6, 3.3, 4.3, 1.7, 3.7, 3.3, 4.2],
 'candidates': [{'card': 'S7',
   'insta_score': 0.419,
   'expected_tricks_dd': 11.16,
   'p_make_contract': 0.96,
   'expected_score_dd': 626,
   'msg': '0 7 0 6 2 8 0 4 2 20 - 1 8 0 6 1 7 0 4 0 17 - 5200300 - 318 - 317'},
  {'card': 'S6',
   'insta_score': 0.419,
   'expected_tricks_dd': 11.16,
   'p_make_contract': 0.96,
   'expected_score_dd': 626,
   'msg': '0 7 0 6 2 8 0 4 2 20 - 1 8 0 6 1 7 0 4 0 17 - 5200300 - 318 - 317'},
  {'card': 'S3',
   'insta_score': 0.419,
   'expected_tricks_dd': 11.16,
   'p_make_contract': 0.96,
   'expected_score_dd': 626,
   'msg': '0 7 0 6 2 8 0 4 2 20 - 1 8 0 6 1 7 0 4 0 17 - 5200300 - 318 - 317'},
  {'card': 'S2',
   'insta_score': 0.419,
   'expected_tricks_dd': 11.16,
   'p_make_contract': 0.96,
   'expected_score_dd': 626,
   'msg': '0 7 0 6 2 8 0 4 2 20 - 1 8 0 6 1 7 0 4 0 17 - 5200300 - 318 - 317'},
  {'card': 'S8',
   'insta_score': 

indeed, the best play is to ruff low.

looking at the samples, we see that East has the `CA` in every sample (this is by inference because underleading an A is very unlikely)

## Analyzing a board played on BBO

In [9]:
# copy-paste from the hand records (in lin format)

lin = 'pn|You,~~M7228oka,~~M72302cm,~~M72316sq|st||md|1S4TKHJD68QC679TKA,S35H479TQKD24TAC8,S2789H3AD379JKC35,|rh||ah|Board 3|sv|e|mb|1C|an|Minor suit opening -- 3+ !C; 11-21 HCP; 12-22 total points|mb|2H|an|Aggressive weak jump overcall -- 6+ !H; 4-10 HCP |mb|d|an|Negative double -- 4+ !S; 7+ HCP; 8+ total points |mb|4H|an|The Law: 10 trump -> game support -- 4+ total points |mb|4S|an|3+ !C; 4+ !S; 16-21 HCP; 17-22 total points|mb|p|mb|p|mb|p|pg||pc|DA|pc|D3|pc|D5|pc|D6|pg||pc|C8|pc|C3|pc|CJ|pc|CA|pg||pc|S4|pc|S5|pc|S8|pc|SJ|pg||pc|H5|pc|HJ|pc|HQ|pc|HA|pg||pc|S2|pc|SA|pc|ST|pc|S3|pg||pc|H2|pc|SK|pc|H4|pc|H3|pg||pc|D8|pc|D2|pc|DJ|pc|S6|pg||pc|SQ|pc|C6|pc|H7|pc|S7|pg||pc|H8|pc|C7|pc|HK|pc|S9|pg||pc|C5|pc|C2|pc|CT|pc|HT|pg||pc|CK|pc|H9|pc|D7|pc|C4|pg||pc|DQ|pc|D4|pc|DK|pc|H6|pg||pc|D9|pc|CQ|pc|C9|pc|DT|pg||'

In [10]:
display_lin(lin)

In [22]:
board = parse_lin(lin)
print(board)

Board(dealer='S', vuln=[False, True], hands=['9872.A3.KJ973.53', '6QAJ.2658.5.2Q4J', 'KT4.J.Q86.AKT976', '53.KQT974.AT42.8'], auction=['1C', '2H', 'X', '4H', '4S', 'PASS', 'PASS', 'PASS'], play=['DA', 'D3', 'D5', 'D6', 'C8', 'C3', 'CJ', 'CA', 'S4', 'S5', 'S8', 'SJ', 'H5', 'HJ', 'HQ', 'HA', 'S2', 'SA', 'ST', 'S3', 'H2', 'SK', 'H4', 'H3', 'D8', 'D2', 'DJ', 'S6', 'SQ', 'C6', 'H7', 'S7', 'H8', 'C7', 'HK', 'S9', 'C5', 'C2', 'CT', 'HT', 'CK', 'H9', 'D7', 'C4', 'DQ', 'D4', 'DK', 'H6', 'D9', 'CQ', 'C9', 'DT'])


In [18]:
card_by_card = CardByCard(*board, models, sampler, False)


In [23]:
await card_by_card.analyze()

analyzing the bidding
Setting seed (Sampling bidding info) from 9872.A3.KJ973.53: 1210459568
Setting seed (Sampling bidding info) from 6QAJ.2658.5.2Q4J: 2412656323
Setting seed (Sampling bidding info) from KT4.J.Q86.AKT976: 3923046009
Setting seed (Sampling bidding info) from 53.KQT974.AT42.8: 485294848
bid 1C value 0.9998 is recommended by NN
CandidateBid(bid=1C  , insta_score=0.9998, expected_score=---, adjust=---)
1C  selected
1C OK NN-value: 1.000
bid 2H value 0.8447 is recommended by NN
CandidateBid(bid=2H  , insta_score=0.8447, expected_score=---, adjust=---)
CandidateBid(bid=1H  , insta_score=0.1511, expected_score=---, adjust=---)
Sampling for aution: ['PAD_START', 'PAD_START', '1C'] trying to find 5000
Found 4279 samples for bidding
 2H   0.845 Samples: 200
bidding_rollout - n_samples:  200
bidding_rollout - finished  (200, 64)
dds took 7.920
[-200. -100.  170. -110.  650.  150.  150. -150.   50. -130.  100. -400.
 -230.  620. -460.  110.  100. -130. -130.   50.   50. -170.   

the engine agrees with the bidding, but didn't like something in the cardplay.

playing `S4` from hand is the first mistake. apparently this play drops almost half a trick on average.

In [24]:
card_by_card.cards['S4'].to_dict()

{'card': 'D8',
 'hcp': [6.7, 8.1],
 'shape': [3.2, 3.9, 3.5, 2.3, 2.3, 6.0, 2.7, 2.0],
 'candidates': [{'card': 'D8',
   'insta_score': 0.116,
   'expected_tricks_dd': 7.58,
   'p_make_contract': 0.0,
   'expected_score_dd': -121,
   'msg': '0 5 0 5 3 5 1 6 4 10 - 0 3 0 5 5 7 0 5 7 13 - 705432 - 2847 - 644'},
  {'card': 'DQ',
   'insta_score': 0.061,
   'expected_tricks_dd': 7.55,
   'p_make_contract': 0.0,
   'expected_score_dd': -123,
   'msg': '0 5 0 5 3 5 1 6 4 10 - 0 3 0 5 5 7 0 5 7 13 - 705432 - 2847 - 644'},
  {'card': 'HJ',
   'insta_score': 0.128,
   'expected_tricks_dd': 7.51,
   'p_make_contract': 0.0,
   'expected_score_dd': -124,
   'msg': '0 5 0 5 3 5 1 6 4 10 - 0 3 0 5 5 7 0 5 7 13 - 705432 - 2847 - 644'},
  {'card': 'CK',
   'insta_score': 0.392,
   'expected_tricks_dd': 7.45,
   'p_make_contract': 0.0,
   'expected_score_dd': -128,
   'msg': '0 5 0 5 3 5 1 6 4 10 - 0 3 0 5 5 7 0 5 7 13 - 705432 - 2847 - 644'},
  {'card': 'S4',
   'insta_score': 0.09,
   'expected_trick

the opening lead of `DA` is interesting. the engine prefers the `HK` and it's the only card it considers.

In [25]:
card_by_card.cards['DA'].to_dict()

{'card': 'DA',
 'quality': 'Good',
 'hcp': [4.2, 1.8, 3.5, 3.5, 2.6, 4.2, 3.0, 3.0, 4.1, 1.4, 2.4, 5.1],
 'shape': [10.8, 7.0, 13.3],
 'candidates': [{'card': 'C8',
   'insta_score': 0.109,
   'expected_tricks_sd': 11.66,
   'p_make_contract': 0.08},
  {'card': 'Hx',
   'insta_score': 0.238,
   'expected_tricks_sd': 11.78,
   'p_make_contract': 0.02},
  {'card': 'HK',
   'insta_score': 0.23,
   'expected_tricks_sd': 11.77,
   'p_make_contract': 0.02}],
 'samples': ['xx.KQT9xx.ATxx.8 QT8xx.x.KQxx.Kxx x.AJ8xxx.Jxx.J9x AKJ9x..98.AQTxxx 0.79423',
  'xx.KQT9xx.ATxx.8 AJxxx.J.KJ9xx.xx x.A8xxx.Qxx.QT9x KQT98.x.8.AKJxxx 0.77794',
  'xx.KQT9xx.ATxx.8 J98xx.A8.K8xx.Qx x.Jxxxx.Q9x.A9xx AKQTx..Jx.KJTxxx 0.77758',
  'xx.KQT9xx.ATxx.8 K9xxx.x.K8xx.AJx x.AJ8xxx.QJx.xxx AQJT8..9x.KQT9xx 0.77259',
  'xx.KQT9xx.ATxx.8 KQ8xx.J.KQxx.Txx J.A8xxxx.J8x.Qxx AT9xx..9x.AKJ9xx 0.77076',
  'xx.KQT9xx.ATxx.8 A9xxx.A8.xxx.QJx x.Jxxxx.Q98x.ATx KQJT8..KJ.K9xxxx 0.76874',
  'xx.KQT9xx.ATxx.8 KJ9xx.A.J98xx.J9 T.J8xxx.K